In [40]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

import tensorflow as tf
import tensorflow_probability as tfp
from sklearn.model_selection import train_test_split


tfd = tfp.distributions
tfb = tfp.bijectors

dtype = tf.float64

%config InlineBackend.figure_format = 'retina'
%matplotlib inline
plt.style.use('ggplot')

In [41]:
data = pd.read_csv('All.csv')

data = data.drop(data.columns[0], axis=1)
data.replace([np.inf, -np.inf], np.nan, inplace=True)
data = data.dropna()

y = data['LUMO']
X = data.loc[:, ['MolWt', 'HvAtMolWt', 'RadE', 'ValE', 'NHOH', 'H', 'NO', 'HAcc', 'HDon', 'Rings', 'AlRing',
                 'AroRing', 'SatRing', 'AroCarb', 'AroHet', 'Heteroatoms', 'RotBond', 'SatCarb', 'SatHet', 'C',
                 'N', 'F', 'Halogens', 'Double', 'Triple', 'MaxAbsPartCh', 'MaxPartCh', 'MinPartCh', 'MinAbsPartCh', 'Group']]
list_numerical = X.columns


# split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [42]:
X_train = X_train.astype('int32')
X_test = X_test.astype('int32')

In [43]:
num_molwt = int(max(X_train['MolWt'])) + 1
num_hvmolwt = int(max(X_train['HvAtMolWt'])) + 1
# num_rade = int(max(X_train['RadE'])) + 1
# num_vale = int(max(X_train['ValE'])) + 1
# num_nhoh = int(max(X_train['NHOH'])) + 1
# num_h = int(max(X_train['H'])) + 1
# num_no = int(max(X_train['NO'])) + 1
# num_hacc = int(max(X_train['HAcc'])) + 1
# num_hdon = int(max(X_train['HDon'])) + 1
# num_rings = int(max(X_train['Rings'])) + 1
# num_alring = int(max(X_train['AlRing'])) + 1
# num_aroring = int(max(X_train['AroRing'])) + 1
# num_satring = int(max(X_train['SatRing'])) + 1
# num_arocarb = int(max(X_train['AroCarb'])) + 1
# num_arohet = int(max(X_train['AroHet'])) + 1
# num_het = int(max(X_train['Heteroatoms'])) + 1
# num_rotbond = int(max(X_train['RotBond'])) + 1
# num_satcarb = int(max(X_train['SatCarb'])) + 1
# num_sathet = int(max(X_train['SatHet'])) + 1
# num_c = int(max(X_train['C'])) + 1
# num_n = int(max(X_train['N'])) + 1
# num_f = int(max(X_train['F'])) + 1
# num_hal = int(max(X_train['Halogens'])) + 1
# num_double = int(max(X_train['Double'])) + 1
# num_triple = int(max(X_train['Triple'])) + 1
# num_maxabs = int(max(X_train['MaxAbsPartCh'])) + 1
# num_maxpart = int(max(X_train['MaxPartCh'])) + 1
# num_minpart = int(max(X_train['MinPartCh'])) + 1
# num_minabs = int(max(X_train['MinAbsPartCh'])) + 1


# num_instructors = max(features_train['instructors']) + 1
# num_departments = max(features_train['departments']) + 1
num_observations = X_train.shape[0]
print('Molecular Weight', num_molwt)
print('Heavy Atom Molecular Weight', num_hvmolwt)

Molecular Weight 3148
Heavy Atom Molecular Weight 2875


In [44]:
class LinearMixedEffectModel(tf.Module):
  def __init__(self):
    # Set up fixed effects and other parameters.
    # These are free parameters to be optimized in E-steps
    self._intercept = tf.Variable(0., name="intercept")            # alpha in eq
    self._effect_service = tf.Variable(0., name="effect_service")  #  beta in eq
    self._stddev_molwt = tfp.util.TransformedVariable(
        1., bijector=tfb.Exp(), name="stddev_molwt")               # sigma in eq
    self._stddev_hvmolwt = tfp.util.TransformedVariable(
        1., bijector=tfb.Exp(), name="stddev_hvmolwt")             # sigma in eq
    # self._stddev_rade = tfp.util.TransformedVariable(
    #     1., bijector=tfb.Exp(), name="stddev_rade")                # sigma in eq
    # self._stddev_vale = tfp.util.TransformedVariable(
    #     1., bijector=tfb.Exp(), name="stddev_vale")                # sigma in eq
    # self._stddev_nhoh = tfp.util.TransformedVariable(
    #     1., bijector=tfb.Exp(), name="stddev_nhoh")                # sigma in eq
    # self._stddev_h = tfp.util.TransformedVariable(
    #     1., bijector=tfb.Exp(), name="stddev_h")                   # sigma in eq
    # self._stddev_no = tfp.util.TransformedVariable(
    #     1., bijector=tfb.Exp(), name="stddev_no")                  # sigma in eq
    # self._stddev_hacc = tfp.util.TransformedVariable(
    #     1., bijector=tfb.Exp(), name="stddev_hacc")                # sigma in eq
    # self._stddev_hdon = tfp.util.TransformedVariable(
    #     1., bijector=tfb.Exp(), name="stddev_hdon")                # sigma in eq
    # self._stddev_rings = tfp.util.TransformedVariable(
    #     1., bijector=tfb.Exp(), name="stddev_rings")               # sigma in eq
    # self._stddev_alring = tfp.util.TransformedVariable(
    #     1., bijector=tfb.Exp(), name="stddev_alring")              # sigma in eq
    # self._stddev_aroring = tfp.util.TransformedVariable(
    #     1., bijector=tfb.Exp(), name="stddev_aroring")             # sigma in eq
    # self._stddev_satring = tfp.util.TransformedVariable(
    #     1., bijector=tfb.Exp(), name="stddev_satring")             # sigma in eq
    # self._stddev_arocarb = tfp.util.TransformedVariable(
    #     1., bijector=tfb.Exp(), name="stddev_arocarb")             # sigma in eq
    # self._stddev_arohet = tfp.util.TransformedVariable(
    #     1., bijector=tfb.Exp(), name="stddev_arohet")              # sigma in eq
    # self._stddev_het = tfp.util.TransformedVariable(
    #     1., bijector=tfb.Exp(), name="stddev_het")                 # sigma in eq
    # self._stddev_rotbond = tfp.util.TransformedVariable(
    #     1., bijector=tfb.Exp(), name="stddev_rotbond")             # sigma in eq
    # self._stddev_satcarb = tfp.util.TransformedVariable(
    #     1., bijector=tfb.Exp(), name="stddev_satcarb")             # sigma in eq
    # self._stddev_sathet = tfp.util.TransformedVariable(
    #     1., bijector=tfb.Exp(), name="stddev_sathet")              # sigma in eq
    # self._stddev_c = tfp.util.TransformedVariable(
    #     1., bijector=tfb.Exp(), name="stddev_c")                   # sigma in eq
    # self._stddev_n = tfp.util.TransformedVariable(
    #     1., bijector=tfb.Exp(), name="stddev_n")                   # sigma in eq
    # self._stddev_f = tfp.util.TransformedVariable(
    #     1., bijector=tfb.Exp(), name="stddev_f")                   # sigma in eq
    # self._stddev_hal = tfp.util.TransformedVariable(
    #     1., bijector=tfb.Exp(), name="stddev_hal")                 # sigma in eq
    # self._stddev_double = tfp.util.TransformedVariable(
    #     1., bijector=tfb.Exp(), name="stddev_double")              # sigma in eq
    # self._stddev_triple = tfp.util.TransformedVariable(
    #     1., bijector=tfb.Exp(), name="stddev_triple")              # sigma in eq
    # self._stddev_maxabs = tfp.util.TransformedVariable(
    #     1., bijector=tfb.Exp(), name="stddev_maxabs")              # sigma in eq
    # self._stddev_maxpart = tfp.util.TransformedVariable(
    #     1., bijector=tfb.Exp(), name="stddev_maxpart")             # sigma in eq
    # self._stddev_minabs = tfp.util.TransformedVariable(
    #     1., bijector=tfb.Exp(), name="stddev_minabs")              # sigma in eq
    # self._stddev_minpart = tfp.util.TransformedVariable(
    #     1., bijector=tfb.Exp(), name="stddev_minpart")             # sigma in eq

  def __call__(self, features):
    model = tfd.JointDistributionSequential([
      # Set up random effects.
      tfd.MultivariateNormalDiag(
          loc=tf.zeros(num_molwt, dtype=tf.dtypes.float32),
          scale_diag=self._stddev_molwt * tf.ones(num_molwt)),
      tfd.MultivariateNormalDiag(
          loc=tf.zeros(num_hvmolwt),
          scale_diag=self._stddev_hvmolwt * tf.ones(num_hvmolwt)),
    #   tfd.MultivariateNormalDiag(
    #       loc=tf.zeros(num_rade),
    #       scale_diag=self._stddev_rade * tf.ones(num_rade)),
    #   tfd.MultivariateNormalDiag(
    #       loc=tf.zeros(num_vale),
    #       scale_diag=self._stddev_vale * tf.ones(num_vale)),
    #   tfd.MultivariateNormalDiag(
    #       loc=tf.zeros(num_nhoh),
    #       scale_diag=self._stddev_nhoh * tf.ones(num_nhoh)),
    #   tfd.MultivariateNormalDiag(
    #       loc=tf.zeros(num_h),
    #       scale_diag=self._stddev_h * tf.ones(num_h)),
    #   tfd.MultivariateNormalDiag(
    #       loc=tf.zeros(num_no),
    #       scale_diag=self._stddev_no * tf.ones(num_no)),
    #   tfd.MultivariateNormalDiag(
    #       loc=tf.zeros(num_hacc),
    #       scale_diag=self._stddev_hacc * tf.ones(num_hacc)),
    #   tfd.MultivariateNormalDiag(
    #       loc=tf.zeros(num_hdon),
    #       scale_diag=self._stddev_hdon * tf.ones(num_hdon)),
    #   tfd.MultivariateNormalDiag(
    #       loc=tf.zeros(num_rings),
    #       scale_diag=self._stddev_rings * tf.ones(num_rings)),
    #   tfd.MultivariateNormalDiag(
    #       loc=tf.zeros(num_alring),
    #       scale_diag=self._stddev_alring * tf.ones(num_alring)),
    #   tfd.MultivariateNormalDiag(
    #       loc=tf.zeros(num_aroring),
    #       scale_diag=self._stddev_aroring * tf.ones(num_aroring)),
    #   tfd.MultivariateNormalDiag(
    #       loc=tf.zeros(num_satring),
    #       scale_diag=self._stddev_satring * tf.ones(num_satring)),
    #   tfd.MultivariateNormalDiag(
    #       loc=tf.zeros(num_arocarb),
    #       scale_diag=self._stddev_arocarb * tf.ones(num_arocarb)),
    #   tfd.MultivariateNormalDiag(
    #       loc=tf.zeros(num_arohet),
    #       scale_diag=self._stddev_arohet * tf.ones(num_arohet)),
    #   tfd.MultivariateNormalDiag(
    #       loc=tf.zeros(num_het),
    #       scale_diag=self._stddev_het * tf.ones(num_het)),
    #   tfd.MultivariateNormalDiag(
    #       loc=tf.zeros(num_rotbond),
    #       scale_diag=self._stddev_rotbond * tf.ones(num_rotbond)),
    #   tfd.MultivariateNormalDiag(
    #       loc=tf.zeros(num_satcarb),
    #       scale_diag=self._stddev_satcarb * tf.ones(num_satcarb)),
    #   tfd.MultivariateNormalDiag(
    #       loc=tf.zeros(num_sathet),
    #       scale_diag=self._stddev_sathet * tf.ones(num_sathet)),
    #   tfd.MultivariateNormalDiag(
    #       loc=tf.zeros(num_c),
    #       scale_diag=self._stddev_c * tf.ones(num_c)),
    #   tfd.MultivariateNormalDiag(
    #       loc=tf.zeros(num_n),
    #       scale_diag=self._stddev_n * tf.ones(num_n)),
    #   tfd.MultivariateNormalDiag(
    #       loc=tf.zeros(num_f),
    #       scale_diag=self._stddev_f * tf.ones(num_f)),
    #   tfd.MultivariateNormalDiag(
    #       loc=tf.zeros(num_hal),
    #       scale_diag=self._stddev_hal * tf.ones(num_hal)),
    #   tfd.MultivariateNormalDiag(
    #       loc=tf.zeros(num_double),
    #       scale_diag=self._stddev_double * tf.ones(num_double)),
    #   tfd.MultivariateNormalDiag(
    #       loc=tf.zeros(num_triple),
    #       scale_diag=self._stddev_triple * tf.ones(num_triple)),
    #   tfd.MultivariateNormalDiag(
    #       loc=tf.zeros(num_maxabs),
    #       scale_diag=self._stddev_maxabs * tf.ones(num_maxabs)),
    #   tfd.MultivariateNormalDiag(
    #       loc=tf.zeros(num_maxpart),
    #       scale_diag=self._stddev_maxpart * tf.ones(num_maxpart)),
    #   tfd.MultivariateNormalDiag(
    #       loc=tf.zeros(num_minabs),
    #       scale_diag=self._stddev_minabs * tf.ones(num_minabs)),
    #   tfd.MultivariateNormalDiag(
    #       loc=tf.zeros(num_minpart),
    #       scale_diag=self._stddev_minpart * tf.ones(num_minpart)),
      # This is the likelihood for the observed.
      lambda effect_molwt, effect_hvatmolwt
            #  effect_rade, effect_vale, effect_nhoh, effect_h, effect_no,
            #  effect_hacc, effect_hdon, effect_rings, effect_alring, effect_aroring, effect_satring,
            #  effect_arocarb, effect_arohet, effect_het, effect_rotbond, effect_satcarb, effect_sathet,
            #  effect_c, effect_n, effect_f, effect_hal, effect_double, effect_triple, effect_maxabs,
            #  effect_maxpart, effect_minpart, effect_minabs
             : tfd.Independent(
          tfd.Normal(
              loc=(self._effect_service * X_train["Group"] +
                  tf.gather(effect_molwt, X_train["MolWt"], axis=-1) +
                  tf.gather(effect_hvatmolwt, X_train["HvAtMolWt"], axis=-1) +
                #   tf.gather(effect_rade, X_train['RadE'], axis=-1) +
                #   tf.gather(effect_vale, X_train['ValE'], axis=-1) +
                #   tf.gather(effect_nhoh, X_train['NHOH'], axis=-1) +
                #   tf.gather(effect_h, X_train['H'], axis=-1) +
                #   tf.gather(effect_no, X_train['NO'], axis=-1) +
                #   tf.gather(effect_hacc, X_train['HAcc'], axis=-1) +
                #   tf.gather(effect_hdon, X_train['HDon'], axis=-1) +
                #   tf.gather(effect_rings, X_train['Rings'], axis=-1) +
                #   tf.gather(effect_alring, X_train['AlRing'], axis=-1) +
                #   tf.gather(effect_aroring, X_train['AroRing'], axis=-1) +
                #   tf.gather(effect_satring, X_train['SatRing'], axis=-1) +
                #   tf.gather(effect_arocarb, X_train['AroCarb'], axis=-1) +
                #   tf.gather(effect_arohet, X_train['AroHet'], axis=-1) +
                #   tf.gather(effect_het, X_train['Heteroatoms'], axis=-1) +
                #   tf.gather(effect_rotbond, X_train['RotBond'], axis=-1) +
                #   tf.gather(effect_satcarb, X_train['SatCarb'], axis=-1) +
                #   tf.gather(effect_sathet, X_train['SatHet'], axis=-1) +
                #   tf.gather(effect_c, X_train['C'], axis=-1) +
                #   tf.gather(effect_n, X_train['N'], axis=-1) +
                #   tf.gather(effect_f, X_train['F'], axis=-1) +
                #   tf.gather(effect_hal, X_train['Halogens'], axis=-1) +
                #   tf.gather(effect_double, X_train['Double'], axis=-1) +
                #   tf.gather(effect_triple, X_train['Triple'], axis=-1) +
                #   tf.gather(effect_maxpart, X_train['MaxPartCh'], axis=-1) +
                #   tf.gather(effect_maxabs, X_train['MaxAbsPartCh'], axis=-1) +
                #   tf.gather(effect_minpart, X_train['MinPartCh'], axis=-1) +
                #   tf.gather(effect_minabs, X_train['MinAbsPartCh'], axis=-1) +
                  self._intercept),
              scale=1.),
              reinterpreted_batch_ndims=1)
    ])

    # To enable tracking of the trainable variables via the created distribution,
    # we attach a reference to `self`. Since all TFP objects sub-class
    # `tf.Module`, this means that the following is possible:
    # LinearMixedEffectModel()(features_train).trainable_variables
    # ==> tuple of all tf.Variables created by LinearMixedEffectModel.
    model._to_track = self
    return model
  
lmm_jointdist = LinearMixedEffectModel()
# Conditioned on feature/predictors from the training data
lmm_train = lmm_jointdist(X_train)

In [45]:
lmm_train.trainable_variables

InvalidArgumentError: {{function_node __wrapped__GatherV2_device_/job:localhost/replica:0/task:0/device:CPU:0}} indices[14119] = 3147 is not in [0, 2875) [Op:GatherV2]

In [ ]:
lmm_train.resolve_graph()

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices